In [ ]:
!wget -q http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishFnt.tgz
!tar -xzvf EnglishFnt.tgz

In [ ]:
import numpy as np
from skimage import io
import os
from PIL import Image
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
def add_margin(pil_img, top, right, bottom, left, color):
  width, height = pil_img.size
  new_width = width + right + left
  new_height = height + top + bottom
  result = Image.new(pil_img.mode, (new_width,   new_height), color)
  result.paste(pil_img, (left, top))
  return result

In [ ]:
dataset=[]
labels=[]
count=0
fnt_path = r"English/Fnt/"
folders= os.listdir(fnt_path)
for i in folders:
  for j in os.listdir(fnt_path+str(i)):
     im_new=add_margin(Image.open(fnt_path+str(i)+'/'+str(j)), 10, 10,10, 10, (255))
     resized_image = im_new.resize((32,32))
     dataset.append(np.array([  np.asarray(resized_image)/255.0 ] ))
     labels.append(count)
  count+=1

In [ ]:
np.array(dataset).shape

(62992, 1, 32, 32)

In [ ]:
dataset = np.array(dataset).reshape(62992,32,32,1)
labels = np.array(labels).reshape(-1,1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
type_encoder = OneHotEncoder()
labels=type_encoder.fit_transform(labels).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset,labels, test_size=0.33, random_state=42)

In [ ]:
y_train.shape

(42204, 62)

In [ ]:
model = Sequential()
# Convolution
model.add(Conv2D(32, (3, 3), input_shape = (32,32,1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
# Pooling
# Convolution
model.add(Conv2D(64, (3, 3),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
# Full connection
#model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dropout(0.5))
model.add(Dense(units = 256, activation = 'relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.5))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 62, activation = 'softmax'))
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [ ]:
history = model.fit(x_train, y_train, batch_size = 4, epochs = 100, validation_data=(x_test, y_test),verbose=1)
model.save('Model.h5')